<h1>Курсовая работа (BIG DATA)<h1>

<h2>Тема: Исследование по загрязнению воздуха в Кыргызской Республики</h2>

<h3>Команда: ЭТЭ</h3>
<h3>Жээнбеков Эрбол</h3>
<h3>Акматов Тариэл</h3>
<h3>Исмаилов Эмирхан</h3>

# (1) Получение данных и перевод их в DataFrame для дальнейшей работы

In [ ]:
import pandas as pd
from datetime import datetime

import predictions

file_path = 'air_pollution_data_kyrgyzstan.xlsx'
air_pollution = pd.read_excel(file_path)

In [ ]:
air_pollution.info()

In [ ]:
print("Ключи digits_dataset: \n{}".format(air_pollution.keys()))

In [ ]:
air_pollution_sorted = air_pollution.sort_values(by='Дата')
air_pollution_sorted.head()

## (2) Сохранение и сортировка данных по месяцам

In [ ]:
air_pollution_sorted["Дата"] = pd.to_datetime(air_pollution_sorted["Дата"], format="%m/%d/%Y, %H:%M:%S")
air_pollution_sorted.head(10)

In [ ]:
monthly_air_pollution_a_cleaneds = {group['Дата'].dt.strftime('%B').iloc[0]: group for _, group in air_pollution_sorted.groupby(air_pollution_sorted['Дата'].dt.month)}

<h4>Я сделал именно такое сохранение так, как в случае сохранения даты в виде строки, отображение в excel происходит в виде "2022-01-31", но сам тип поля "Дата" становится object</h4>

In [ ]:
for month_name, group in monthly_air_pollution_a_cleaneds.items():
    file_name_month = f"air_pollution_data_kyrgyzstan_{month_name.lower()}.xlsx"
    group["Дата"] = pd.to_datetime(group["Дата"], format="%Y-%m-%d")
    group.to_excel(file_name_month, index=False)

<h4>Вот здесь ниже я привожу пример одной из таблиц для того чтобы наглядно показать сохранение данных в виде даты</h4>

In [ ]:
april_path = 'air_pollution_data_kyrgyzstan_april.xlsx'
april_air = pd.read_excel(april_path)
april_air.head() 

In [ ]:
april_air["Дата"] = pd.to_datetime(april_air["Дата"], format="%Y-%m-%d")
april_air.info()

## Сохранение и сортировка данных по городам

In [ ]:
grouped_by_city = air_pollution_sorted.groupby('Город')

In [ ]:
for city, group in grouped_by_city:
    if city == 'Бишкек':
        city = 'Bishkek'
    if city == 'Ош':
        city = 'Osh'
    if city == 'Джалал-Абад':
        city = 'Djalal-abad'
    if city == 'Каракол':
        city = 'Karakol'
    if city == 'Талас':
        city = 'Talas'
    file_name_city = f"air_pollution_data_kyrgyzstan_{city.lower()}.xlsx"
    group.to_excel(file_name_city, index=False)

In [ ]:
talas_path = 'air_pollution_data_kyrgyzstan_talas.xlsx'
talas_air = pd.read_excel(talas_path)
talas_air.head() 

In [ ]:
talas_air["Дата"] = pd.to_datetime(talas_air["Дата"], format="%Y-%m-%d")
talas_air.info()

In [ ]:
air_pollution_sorted.info()

In [ ]:
# Обработка пустых значений
air_pollution_cleaned = air_pollution_sorted.ffill()  # Заполнение пропущенных значений предыдущими значениями
# Удаление дубликатов 
air_pollution_cleaned = air_pollution_cleaned.drop_duplicates()

print(air_pollution_cleaned.info())


In [ ]:
air_pollution_cleaned.tail()

In [ ]:
air_pollution_cleaned['PM10'] = pd.to_numeric(air_pollution_cleaned['PM10'], errors='coerce')
print(air_pollution_cleaned['PM10'])
print(air_pollution_cleaned['PM10'].dtype)

In [ ]:
air_pollution_cleaned.dropna()['PM10'].tail(10)

In [ ]:

# Предполагая, что ваш DataFrame назван 'air_pollution_cleaned'
# Замените 'air_pollution_cleaned' на фактическое имя вашего DataFrame, если оно отличается

# Выберите столбцы типа float
float_columns = air_pollution_cleaned.select_dtypes(include=['float64']).columns

# Проверьте каждый столбец float на наличие значений, не являющихся float
for column in float_columns:
    non_float_values = air_pollution_cleaned[~air_pollution_cleaned[column].apply(lambda x: isinstance(x, (int, float)))][column]
    
    if not non_float_values.empty:
        print(f"Столбец '{column}' содержит значения, не являющиеся типом float:")
        print(non_float_values)
        print("\n")
    else:
        print(f"Столбец '{column}' содержит только значения типа float.\n")


In [ ]:
air_pollution_cleaned.info()
air_pollution_cleaned.set_index('Дата', inplace=True)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 18))  # Adjust the figure size as needed
std_values = air_pollution_cleaned['PM2.5'].rolling(window=10).std()
mean_values = air_pollution_cleaned['PM2.5'].rolling(window=10).mean()
# Plot for Уровень SO2
plt.subplot(3, 1, 1)
plt.plot(air_pollution_cleaned.index,air_pollution_cleaned['PM2.5'], marker='o', linestyle='-', color='orange')
plt.title('Изменение уровня PM2.5 во времени')
plt.xlabel('Дата')
plt.ylabel('Уровень PM2.5')
plt.grid(True)

# Plot for Стандартное отклонение
plt.subplot(3, 1, 2)
plt.plot(air_pollution_cleaned.index, std_values, linestyle='--', color='red')
plt.title('Изменение стандартного отклонения PM2.5 во времени')
plt.xlabel('Дата')
plt.ylabel('Стандартное отклонение PM2.5')
plt.grid(True)

# Plot for Среднее значение
plt.subplot(3, 1, 3)
plt.plot(air_pollution_cleaned.index, mean_values, linestyle='--', color='blue')
plt.title('Изменение среднего значения PM2.5 во времени')
plt.xlabel('Дата')
plt.ylabel('Среднее значение PM2.5')
plt.grid(True)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 18))  # Adjust the figure size as needed
std_values = air_pollution_cleaned['SO2'].rolling(window=10).std()
mean_values = air_pollution_cleaned['SO2'].rolling(window=10).mean()
# Plot for Уровень SO2
plt.subplot(3, 1, 1)
plt.plot(air_pollution_cleaned.index,air_pollution_cleaned['SO2'], marker='o', linestyle='-', color='orange')
plt.title('Изменение уровня SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Уровень SO2')
plt.gca().set_facecolor('white')
plt.grid(True)

# Plot for Стандартное отклонение
plt.subplot(3, 1, 2)
plt.plot(air_pollution_cleaned.index, std_values, linestyle='--', color='red')
plt.title('Изменение стандартного отклонения SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Стандартное отклонение SO2')
plt.gca().set_facecolor('white')
plt.grid(True)

# Plot for Среднее значение
plt.subplot(3, 1, 3)
plt.plot(air_pollution_cleaned.index, mean_values, linestyle='--', color='blue')
plt.title('Изменение среднего значения SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Среднее значение SO2')
plt.gca().set_facecolor('white')
plt.grid(True)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 18))  # Adjust the figure size as needed
std_values = air_pollution_cleaned['PM10'].rolling(window=10).std()
mean_values = air_pollution_cleaned['PM10'].rolling(window=10).mean()
# Plot for Уровень SO2
plt.subplot(3, 1, 1)
plt.plot(air_pollution_cleaned.index,air_pollution_cleaned['PM10'], marker='o', linestyle='-', color='orange')
plt.title('Изменение уровня PM10 во времени')
plt.xlabel('Дата')
plt.ylabel('Уровень PM10')
plt.gca().set_facecolor('white')
plt.grid(True)

# Plot for Стандартное отклонение
plt.subplot(3, 1, 2)
plt.plot(air_pollution_cleaned.index, std_values, linestyle='--', color='red')
plt.title('Изменение стандартного отклонения PM10 во времени')
plt.xlabel('Дата')
plt.ylabel('Стандартное отклонение PM10')
plt.gca().set_facecolor('white')
plt.grid(True)

# Plot for Среднее значение
plt.subplot(3, 1, 3)
plt.plot(air_pollution_cleaned.index, mean_values, linestyle='--', color='blue')
plt.title('Изменение среднего значения PM10 во времени')
plt.xlabel('Дата')
plt.ylabel('Среднее значение PM10')
plt.gca().set_facecolor('white')
plt.grid(True)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


<h3>Давайте возьмем Бишкек для примера!</h3>

In [ ]:
bishkek_path = 'air_pollution_data_kyrgyzstan_bishkek.xlsx'
bishkek_air = pd.read_excel(bishkek_path)
bishkek_air.head() 

In [ ]:
bishkek_air.info()
bishkek_air.set_index('Дата', inplace=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 18))  # Adjust the figure size as needed
std_values = bishkek_air['SO2'].rolling(window=10).std()
mean_values = bishkek_air['SO2'].rolling(window=10).mean()
# Plot for Уровень SO2
plt.subplot(3, 1, 1)
plt.plot(bishkek_air.index, bishkek_air['SO2'], marker='o', linestyle='-', color='orange')
plt.title('Изменение уровня SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Уровень SO2')
plt.grid(True)

# Plot for Стандартное отклонение
plt.subplot(3, 1, 2)
plt.plot(bishkek_air.index, std_values, linestyle='--', color='red')
plt.title('Изменение стандартного отклонения SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Стандартное отклонение SO2')
plt.grid(True)

# Plot for Среднее значение
plt.subplot(3, 1, 3)
plt.plot(bishkek_air.index, mean_values, linestyle='--', color='blue')
plt.title('Изменение среднего значения SO2 во времени')
plt.xlabel('Дата')
plt.ylabel('Среднее значение SO2')
plt.grid(True)

plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:

# Assuming 'Дата' column is datetime type
bishkek_air.info()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar

# Assuming 'Дата' column is datetime type
bishkek_air['Month'] = bishkek_air.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
bishkek_air['Month'] = bishkek_air['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by month and calculate the mean, then sort by month order
monthly_mean = bishkek_air.groupby('Month')['SO2'].mean().reindex(month_names)

# Plotting the bar graph
plt.figure(figsize=(10, 6))
monthly_mean.plot(kind='bar', color='green', alpha=0.7)
plt.title('Средний уровень SO2 по месяцам (отсортировано)')
plt.xlabel('Месяц')
plt.ylabel('Средний уровень SO2')
plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import calendar

# Assuming 'Дата' column is datetime type
bishkek_air['Month'] = bishkek_air.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
bishkek_air['Month'] = bishkek_air['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by month and calculate the mean, then sort by month order
monthly_mean = bishkek_air.groupby('Month')['PM2.5'].mean().reindex(month_names)

# Plotting the bar graph
plt.figure(figsize=(10, 6))
monthly_mean.plot(kind='bar', color='green', alpha=0.7)
plt.title('Средний уровень PM2.5 по месяцам (отсортировано)')
plt.xlabel('Месяц')
plt.ylabel('Средний уровень PM2.5')
plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar

# Assuming 'Дата' column is datetime type
bishkek_air['Month'] = bishkek_air.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
bishkek_air['Month'] = bishkek_air['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by month and calculate the mean, then sort by month order
monthly_mean = bishkek_air.groupby('Month')['PM10'].mean().reindex(month_names)

# Plotting the bar graph
plt.figure(figsize=(10, 6))
monthly_mean.plot(kind='bar', color='green', alpha=0.7)
plt.title('Средний уровень PM10 по месяцам (отсортировано)')
plt.xlabel('Месяц')
plt.ylabel('Средний уровень PM10')
plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


Средние показатели SO2, PM2.5 и PM10 по городам

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar

# Assuming 'Дата' column is datetime type
air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10 = air_pollution_cleaned.groupby('Город')['PM10'].mean().sort_values()

# Plotting the bar graph
plt.figure(figsize=(12, 8))
city_mean_pm10.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень PM10 по городам')
plt.xlabel('Средний уровень PM10')
plt.yticks(range(len(city_mean_pm10)), city_mean_pm10.index)  # Set y-axis ticks to city names
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar

# Assuming 'Дата' column is datetime type
air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10 = air_pollution_cleaned.groupby('Город')['SO2'].mean().sort_values()

# Plotting the bar graph
plt.figure(figsize=(12, 8))
city_mean_pm10.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень SO2 по городам')
plt.xlabel('Средний уровень SO2')
plt.yticks(range(len(city_mean_pm10)), city_mean_pm10.index)  # Set y-axis ticks to city names
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import calendar

# Assuming 'Дата' column is datetime type
air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10 = air_pollution_cleaned.groupby('Город')['PM2.5'].mean().sort_values()

# Plotting the bar graph
plt.figure(figsize=(12, 8))
city_mean_pm10.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень PM2.5 по городам')
plt.xlabel('Средний уровень PM2.5')
plt.yticks(range(len(city_mean_pm10)), city_mean_pm10.index)  # Set y-axis ticks to city names
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import calendar

# Assuming 'Дата' column is datetime type
air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Filter data for winter months (December, January, February)
winter_data = air_pollution_cleaned[air_pollution_cleaned['Month'].isin(['Dec', 'Jan', 'Feb'])]

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10_winter = winter_data.groupby('Город')['PM10'].mean().sort_values()

# Plotting the bar graph for winter months
plt.figure(figsize=(12, 8))
city_mean_pm10_winter.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень PM10 по городам (зимние месяцы)')
plt.xlabel('Средний уровень PM10')
plt.ylabel('Город')
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import calendar

# Assuming 'Дата' column is datetime type
air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Filter data for winter months (December, January, February)
winter_data = air_pollution_cleaned[air_pollution_cleaned['Month'].isin(['Dec', 'Jan', 'Feb'])]

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10_winter = winter_data.groupby('Город')['PM2.5'].mean().sort_values()

# Plotting the bar graph for winter months
plt.figure(figsize=(12, 8))
city_mean_pm10_winter.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень PM2.5 по городам (зимние месяцы)')
plt.xlabel('Средний уровень PM2.5')
plt.ylabel('Город')
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import calendar

air_pollution_cleaned['Month'] = air_pollution_cleaned.index.month

# Map month numbers to short month names
month_names = [calendar.month_abbr[i] for i in range(1, 13)]
air_pollution_cleaned['Month'] = air_pollution_cleaned['Month'].map({i: month_names[i-1] for i in range(1, 13)})

# Filter data for winter months (December, January, February)
winter_data = air_pollution_cleaned[air_pollution_cleaned['Month'].isin(['Dec', 'Jan', 'Feb'])]

# Group by city and calculate the mean 'PM10' for each city
city_mean_pm10_winter = winter_data.groupby('Город')['SO2'].mean().sort_values()

# Plotting the bar graph for winter months
plt.figure(figsize=(12, 8))
city_mean_pm10_winter.plot(kind='barh', color='blue', alpha=0.7)
plt.title('Средний уровень SO2 по городам (зимние месяцы)')
plt.xlabel('Средний уровень SO2')
plt.ylabel('Город')
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.show()


<h4>Вычисление коэффициентов корреляции:</h4>

<p>В данном контексте проводится расчет коэффициента корреляции Пирсона для оценки степени линейной взаимосвязи между парами переменных. Этот коэффициент позволяет определить, насколько две переменные изменяются вместе, следуя линейному тренду.</p>

<h4>Визуализация корреляций:</h4>

<p>Для наглядного представления взаимосвязей между различными параметрами используется построение тепловой карты корреляций. Такая визуализация позволяет легко определить, есть ли положительная или отрицательная корреляция между парами переменных, и предоставляет общий обзор структуры взаимосвязей в данных.</p>

In [ ]:
air_pollution_cleaned.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'Город', 'Демография', 'Здоровье населения', 'Экономические показатели', and 'Month' are categorical columns
categorical_columns = [
    "Город",
    "Демография",
    "Здоровье населения",
    "Экономические показатели",
    "Month",
]
air_pollution_encoded = pd.get_dummies(air_pollution_cleaned, columns=categorical_columns)

# Calculate the correlation matrix
correlation_matrix = air_pollution_encoded.corr(method="pearson")

# Increase figure size and adjust layout
plt.figure(figsize=(20, 15))  # Adjust width and height to your preference

# Customize colormap and annotations
cmap = sns.diverging_palette(220, 20, as_cmap=True)
fmt = ".2f"  # Display correlation values with 2 decimal places
annot_kws = {"size": 10}  # Adjust annotation font size if needed

# Generate the heatmap with improved aesthetics
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap=cmap,
    fmt=fmt,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},  # Adjust colorbar size
    annot_kws=annot_kws,
)

# Title and axis labels
plt.title("Матрица корреляции", fontsize=24)  # Adjust title font size
plt.xticks(rotation=45, ha="right", fontsize=12)  # Rotate and style x-axis labels
plt.yticks(rotation=0, fontsize=12)  # Style y-axis labels

# Show the plot
plt.tight_layout()  # Improve overall layout
plt.show()


<h4>Выделение зависимых переменных:<h4>

Зависимая переменная: Уровень загрязнения воздуха (PM2.5, PM10, SO2). Это переменная, которую мы хотим предсказать в нашей модели.
Выбор независимых переменных:

Независимые переменные (факторы), которые могут влиять на уровень загрязнения воздуха. Некоторые из них:
Демографические показатели.
Экономические показатели.
Здоровье населения.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

categorical_columns = ['Город', 'Демография', 'Здоровье населения', 'Экономические показатели', 'Month']

float_columns = ['PM2.5', 'PM10', 'SO2']

In [213]:
X = air_pollution_cleaned[float_columns]
y = air_pollution_cleaned[categorical_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [216]:
print("Форма массива X_train: {}".format(X_train.shape))
print("Форма массива y_train: {}".format(y_train.shape))

Форма массива X_train: (45, 3)
Форма массива y_train: (45, 5)


In [217]:
print("Форма массива X_test: {}".format(X_test.shape))
print("Форма массива y_trest: {}".format(y_test.shape))

Форма массива X_test: (15, 3)
Форма массива y_trest: (15, 5)


In [218]:
print(X_train.isnull().sum())

PM2.5    0
PM10     1
SO2      0
dtype: int64


In [219]:
X_train = X_train.dropna()

In [220]:
print(X_train.isnull().sum())

PM2.5    0
PM10     0
SO2      0
dtype: int64


In [221]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (44, 3)
y_train shape: (45, 5)


In [222]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44 entries, 2022-09-30 to 2022-09-30
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PM2.5   44 non-null     float64
 1   PM10    44 non-null     float64
 2   SO2     44 non-null     float64
dtypes: float64(3)
memory usage: 1.4 KB


In [223]:
from sklearn.ensemble import RandomForestRegressor

y_train_encoded = pd.get_dummies(y_train, columns=categorical_columns)
model = RandomForestRegressor(n_estimators=100000, random_state=42)

# Now, you can train your model with the one-hot encoded data
model.fit(X_train, y_train_encoded[:44])


KeyboardInterrupt: 

In [ ]:
model.score(X_train, y_train_encoded[:44])

In [ ]:
y_test.info()

In [ ]:
y_test_encoded = pd.get_dummies(y_test, columns=categorical_columns)
y_test_encoded.info()

In [ ]:
X_test.info()

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X_test)
predictions